# Text to SQL - Question generation and testing for the Hack Zurich

In [ ]:
!git clone https://github.com/statistikZH/statbot.git
# !git pull

In [ ]:
%cd statbot/hackathon_hackzurich/
!pwd

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
import os
import random
from typing import List, Tuple
import json

import pandas as pd
import sqlalchemy
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM



If you plan to add views to generate questions, you should add rows to the csv file in input_data/INDICATORS_VIEW.csv. Do not fill all columns (indicator_id, a short description, question type (as 1), name of the view and column name of the view value).


In [ ]:
NB_SAMPLES = 1 # number of samples per type (default: 1)

# Get Zurich and Basel data

In [ ]:
# Data paths
INPUT_FOLDER = 'input_data'
INDICATORS_FILE = 'EN_INDICATORS.csv'
INDICATORS_VALUES_FILE = 'EN_INDICATOR_VALUES.csv'
INDICATORS_VIEWS_FILE = 'INDICATORS_VIEWS.csv'
SPATIALUNIT_FILE = 'EN_T_SPATIALUNIT.csv'

# Generation of natural language questions & SQL Statements
Read in a dataset with indicator metadata and short descriptions which can be used to generate questions.

In [ ]:
indicators = pd.read_csv(os.path.join(INPUT_FOLDER, INDICATORS_FILE))
indicators.columns = [c.lower() for c in indicators.columns]
indicators = indicators.rename(columns={"name": "indicator_name"})

In [ ]:
descriptions = pd.read_csv(os.path.join(INPUT_FOLDER, INDICATORS_VIEWS_FILE))
indicators = indicators.merge(descriptions, on='indicator_id', how='left')

### Create a single dataset with all information
 Create a combined dataset containing all the information (values, indicator labels, spatial unit labels)  to make the generation of the question / sql pairs easier. 

In [ ]:
indicators_values = pd.read_csv(os.path.join(INPUT_FOLDER, INDICATORS_VALUES_FILE))
indicators_values.columns = [c.lower() for c in indicators_values.columns]
gp_data = indicators_values.merge(indicators, on='indicator_id', how='left')

In [ ]:
spatial_units = pd.read_csv(os.path.join(INPUT_FOLDER, SPATIALUNIT_FILE))
spatial_units.columns = [c.lower() for c in spatial_units.columns]
spatial_units = spatial_units.rename(columns={'name': 'municipality_name'})

In [ ]:
gp_data = gp_data.merge(spatial_units, on='spatialunit_id', how='left')

### Create a dataset with random values, years and municipalities per indicator

 To fill in the questions and queries, random values, years and municipalities are drawn for each indicator. These are then integrated into the templates dynamically.


In [ ]:
spatial_units = spatial_units[['spatialunit_id', 'municipality_name']]
gp_data = gp_data[gp_data['type_id']==1]
gp_data = gp_data[gp_data['question_type']<=2]
grouping_columns = ['indicator_id', 'indicator_name', 'short_description', 'question_type']
grouped_samples = gp_data.groupby(by=grouping_columns)

sample_columns = grouping_columns + ['value', 'year', 'spatialunit_id', 'view_name', 'view_column_value']
samples = grouped_samples.sample(n=NB_SAMPLES).reset_index()[sample_columns]
samples = samples.merge(spatial_units, on='spatialunit_id', how='left')
samples

In [ ]:
def create_question_query(sample: pd.Series) -> Tuple[List[str], List[str]]:
    '''
    From a sample values, create 7 questions and 7 associated queries
    Args:
        - sample: row of random observations values
    Return:
        - questions: list of question generated with the values from sample
        - queries: : list of queries generated with the values from sample
    '''

    indicator, view, view_column, random_value, indicator_id, indicator_year, municipality = sample[
        ['short_description', 'view_name', 'view_column_value', 'value', 'indicator_id', 'year', 'municipality_name']
    ].values
    
    questions = [
        f"How high is the {indicator} in {municipality} in the year {indicator_year}?", # 0
        f"Which municipality has the highest {indicator}?", # 1
        f"Which municipality has the minimum {indicator}?", # 2
        f"What are the highest, lowest and average {indicator}?", # 3
        f"How many municipalities have a {indicator} higher than {random_value} per year?", # 4
        f"How high is the total {indicator} in the Canton Zurich in the year {indicator_year}?", # 5
        f"Which region had the lowest {indicator} in the year {indicator_year}?" # 6
    ]
    
    queries = [
        # 0
        f"SELECT T1.{view_column} \
        FROM {view} AS T1 \
        JOIN spatialunit AS T2 ON T1.spatialunit_id = T2.spatialunit_id \
        WHERE T2.name LIKE '{municipality}' AND T1.year = {indicator_year}", 
        
        # 1
        f"SELECT T2.name \
        FROM spatialunit AS T2 \
        JOIN {view} AS T1 ON T2.spatialunit_id = T1.spatialunit_id \
        ORDER BY T1.{view_column} DESC LIMIT 1",
        
        # 2
        f"SELECT T2.name \
        FROM spatialunit AS T2 \
        JOIN {view} AS T1 ON T2.spatialunit_id = T1.spatialunit_id \
        ORDER BY T1.{view_column} ASC LIMIT 1",
        
        # 3
        f"SELECT MAX(T1.{view_column}::numeric), \
        MIN(T1.{view_column}::numeric), AVG(T1.{view_column}::numeric) \
        FROM {view} AS T1",
        
        # 4
        f"SELECT T1.year, COUNT(*) \
        FROM {view} AS T1 \
        JOIN spatialunit AS T2 ON T1.spatialunit_id = T2.spatialunit_id \
        WHERE T1.{view_column}::numeric > {random_value} AND T2.type_id = 1 \
        GROUP BY T1.year",
        
        # 5
        f"SELECT T1.{view_column} FROM {view} AS T1 \
        JOIN spatialunit AS T2 ON T1.spatialunit_id = T2.spatialunit_id \
        WHERE T2.type_id = 8 AND T1.year = {indicator_year}",
        
        # 6
        f"SELECT T2.name \
        FROM {view} AS T1 \
        JOIN spatialunit AS T2 ON T1.spatialunit_id = T2.spatialunit_id \
        WHERE T2.type_id = 4 AND T1.year = {indicator_year} \
        ORDER BY T1.{view_column} \
        LIMIT 1"
    ]
    return questions, queries

In [ ]:
all_questions, all_queries = [], []
for _, sample in samples.iterrows():   
    questions, queries = create_question_query(sample)
    all_questions.extend(questions)
    all_queries.extend(queries)

In [ ]:
df = pd.DataFrame({'questions': all_questions, 'queries': all_queries})
df.to_csv("questions_queries_python.csv", index=False)

## Test the SQL queries on Postgres

Run this if you want to test the queries in postgres (you will need an up to date postgres database).e



In [ ]:
!pip install sqlalchemy

In [ ]:
import psycopg2
import sqlalchemy

DB_USER = 'YOUR_DB_USER'
DB_PW = 'YOUR_PASSWORD'
DB_HOST = 'YOUR_DB_HOST'
DB_PORT = 'YOUR_DB_PORT'
DB_SCHEMA = 'YOUR_DB_SCHEMA'
DB = 'YOUR_DB'
DRIVERNAME = "postgresql"

engine = sqlalchemy.create_engine(
    sqlalchemy.engine.url.URL(
        drivername=DRIVERNAME,
        username=DB_USER,
        password=DB_PW,
        host=DB_HOST,
        port=DB_PORT,
        database=DB,
    ),
    echo_pool=True,
)
print("connecting with engine " + str(engine))
connection = engine.connect()

In [ ]:
df = pd.read_csv("questions_queries_python.csv")

In [ ]:
def run_query(df: pd.DataFrame, query_number: int) -> None:
    '''
    Run a query on the database and prints the associted question and answer
    Args:
        - df: dataframe with all random samples
        - query_number: index of the row of (question, query) to select
    '''
    question = df['questions'].iloc[query_number]
    query = df['queries'].iloc[query_number]
    answer = connection.execute(query)
    
    print(f"Question {query_number}: {question}")
    print(f"Answer {query_number}: {[r for r in answer]}\n")

### Run one query

In [ ]:
run_query(df, query_number=6)

### Run all queries

In [ ]:
for i in range(len(df)):
    run_query(df, i)
    if i>20:
        break

## Generate paraphrases

ML-Model taken from:
* [https://github.com/Vamsi995/Paraphrase-Generator](https://github.com/Vamsi995/Paraphrase-Generator)
* [https://huggingface.co/Vamsi/T5_Paraphrase_Paws](https://huggingface.co/Vamsi/T5_Paraphrase_Paws)

In [ ]:

df = pd.read_csv("questions_queries_python.csv")
tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws" , use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

In [ ]:
def generate_paraphrases(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Script for creating paraphrases and thus expanding the q&a pairs generated through the first script
    At the time of writing it generates from 90 initial q&a pairs, around 

    ML-Model taken from https://github.com/Vamsi995/Paraphrase-Generator
    https://huggingface.co/Vamsi/T5_Paraphrase_Paws

    Version 0.1.1 - 15.09.2021
    Christian Ruiz - Statistisches Amt Kanton Zürich
    CC0
    
    History 
    Version 0.1.2 -15.09.2021 - Umlaut-corrections for the SQL-queries
    Version 0.1.1 -15.09.2021 - First version public
    '''
    output_df = df

    for i in range(len(df)):
        sentence=df['questions'].iloc[i]
        print(i, " ", sentence)
        text =  "paraphrase: " + sentence + " </s>"

        encoding = tokenizer.encode_plus(text, pad_to_max_length=True, return_tensors="pt")
        input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]

        outputs = model.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            max_length=256,
            do_sample=True,
            top_k=120,
            top_p=0.95,
            early_stopping=True,
            num_return_sequences=8
        )

        for output in outputs:
            line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            output_df = output_df.append({'questions': line, 'queries': df['queries'].iloc[i]}, ignore_index=True)
    
    return output_df

In [ ]:
output_df = generate_paraphrases(df)

In [ ]:
# Adjustments
output_df = output_df[['questions','queries']]
output_df['questions'] = output_df['questions'].str.lower()
output_df = output_df.drop_duplicates()

# Replace the Umlaut in the SQL-queries for ValueNet
output_df['queries'] = output_df['queries'].str.replace("ü", "ue")
output_df['queries'] = output_df['queries'].str.replace("ä", "ae")
output_df['queries'] = output_df['queries'].str.replace("ö", "oe")

# Save paraphrases
output_df.to_csv("questions_queries_paraphrases.csv", index=False)

## Convert to valunet preprocessing format

Valuenet errors when tokenising the SQL with ::numeric, hence we remove it here. If you query a PostgreSQL database again with these queries, be mindful that you might need the ::numeric.



In [ ]:
df = pd.read_csv("questions_queries_paraphrases.csv")

In [ ]:
def df_to_spider_format(df: pd.DataFrame) -> list:
    handmade_data = []
    
    for _, row in df.iterrows():
        query = row['queries'].replace('::numeric', '')
        row_dict = {
            'db_id': 'hack_zurich',
            'query': query,
            'question': row['questions']
        }
        handmade_data.append(row_dict)
    return handmade_data

In [ ]:
handmade_data = df_to_spider_format(df)

## Save question_queries in the format required by valuenet

In [ ]:
TRAIN_RATIO = 0.8
training_samples = int(len(handmade_data)*TRAIN_RATIO)
train_data = handmade_data[:training_samples]
dev_data = handmade_data[training_samples:]

with open('handmade_data_train.json', 'w') as outfile:
    json.dump(train_data, outfile)

with open('handmade_data_dev.json', 'w') as outfile:
    json.dump(dev_data, outfile)

len(train_data), len(dev_data)

## Preprocessing

If you plan to train Valuenet with this data, you can follow the steps explained in [preprocess_custom_data_hack_zurich_2021.ipynb](https://github.com/hack-with-admin-ch/HackZurich2021/blob/main/notebooks/preprocess_custom_data_hack_zurich_2021.ipynb).
